# Reading ISMN data
This example shows the basic functionality to read data downloaded from the Internatinal Soil Moisture Network (ISMN).
The data is accessible at `http://ismn.geo.tuwien.ac.at` after registration.

For this tutorial, data for the networks 'REMEDHUS', 'SMOSMANIA', 'FMI', 'WEGENERNET', 'GTK' and 'VAS' between 2009-08-04 and 2020-12-12 was downloaded.

The suggested class for reading is called `ISMN_Interface`. It provides functions to access single networks, stations and sensors and the measured time series for each sensor. The `ISMN_Interface` class takes either the directory where the extracted ISMN files are stored, or a zip archive of the data directly (reading from zip is significantly slower).
To read only a selection of networks stored in the passed data path, you can pass a list of network names.

In [1]:
from ismn.interface import ISMN_Interface
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Enter the path to your ISMN data
path_to_ismn_data = r"C:\Temp\delete_me\ismn\stick\Data_separate_files_20090804_20201212.zip"

ismn_data = ISMN_Interface(path_to_ismn_data, network=['REMEDHUS', 'SMOSMANIA', 'FMI', 'WEGENERNET', 'GTK', 'VAS', 'RSMN'])

Found existing ismn metadata in C:\Temp\delete_me\ismn\stick\python_metadata\Data_separate_files_20090804_20201212.csv.


The following command will initialise the reader. The first time this is done, metadata for each sensor will we collected. This will iterate through all files and detect e.g. station names, time coverage, measurement depths etc. Metadata collection can take a few minutes and will start multiple parallel processes (depending on the available ressources). By default the `python_metadata` (which contains the collected metadata as a .csv file) will be placed in the passed root directory. The next time the reader is created it will use `python_metadata` (if it is found) instead of generating it again.

Note: When changing the data (e.g. if folders are added or removed from the data collection) make sure to delete the `python_metadata` folder to re-generate for the new data.

You can define a different path, where the metadata is stored, resp. looked for by passing a `meta_path` when initialsing `ISMN_Interface`. You can also define a list of network names that are considered when reading the data (this will not affect metadata generation, which is performed for ALL networks in `data_path`). Passing `keep_loaded_data=True` means that all time series, once read, will be kept in memory for faster subsequent access. This can fill up your memory and is only recommended for small data samples.

In this example we use the default metadata path.

## Accessing data components
### Network Collection

`ISMN_Interface` contains a `collection` of active ISMN networks. The collection lists the network name and names of all stations in a network (either all Networks if no network(s) was/were specified during initialisation or the selected networks).

In [2]:
collection = ismn_data.collection
collection

The collection also contains a grid object that contains the locations of all **stations** in all active networks. For more details see https://github.com/TUW-GEO/pygeogrids

In [3]:
import pandas as pd
grid = collection.grid
gpis, lons, lats = grid.get_grid_points()
display(pd.DataFrame(index=pd.Index(gpis, name='gpi'), data={'lon': lons, 'lat': lats}).T)

gpi
lon
lat


Using the GPI or coordinates, a station from **all** stations in **all** networks in the collection can be selected.

In [4]:
station, dist = collection.get_nearest_station(27.0, 68.0)
assert collection.station4gpi(0) == station # same result when selecting with GPI

print(f"Station '{station.name}' at Lon: {station.lon}°, Lat: {station.lat}°")
station

ValueError: Index not found in loaded grid: [4294967295]

### Network

A single network from the collection can be accessed via its name.

In [ ]:
network = collection['SMOSMANIA']
network

### Station

A network consists of multiple stations, multiple variables can be measured by different sensors at a station:

In [ ]:
station = network.stations['SaintFelixdeLauragais']
station

Similar as the single sensors, each station has a metadata attribute. The station metadata contains all meta variables from all sensors that measure at the station. Formatting options for MetaData are either as a DataFrame (`to_pd()`) or as a dictionary (`to_dict()`) of form:

```
{name: [(value, depth_from, depth_to), ...], ...}
```

In [ ]:
from pprint import pprint
pprint(station.metadata.to_dict())

### Sensor

Accessing sensors at a station works similar to accessing stations in a network. By default the name is created from the instrument type, the measurued variable and the depth layer that the senosor measures in.

In [ ]:
sensor = station['ThetaProbe-ML2X_soil_moisture_0.050000_0.050000']
sensor

A data file is assigned to each sensor, that contains the sensor variable time series and quality flags. 

In [ ]:
ts = sensor.read_data()
ax = ts.plot(figsize=(12,4))
ax.set_xlabel("Time [year]")
ax.set_ylabel("Soil Moisture [$m^3 m^{-3}$]")
display(ts)

Additionally, metadata is assigned to each sensor. Some metadata is sensor specific (e.g. soil properties, time series range), some depends on the location of the station and is therefore shared by multiple sensors (landcover and climate classes etc.).
The first value in each Variable is the name of the metadata variable, the second the actual value for the variable. The third value (tuple) is the depth that the value applies to (for soil properties multiple layers are provided together with the ISMN data, during metadata generation the best matching depth for a sensor is selected).

In [ ]:
sensor.metadata.to_pd()

## Other functions

### Find network for a specific station
`ISMN_Interface` provides a function to find the network when only the name of a station is known.

In [ ]:
ismn_data.network_for_station('SAA111', name_only=False)

### Read via index

You can filter the dataset a priori and get ids of sensors that measure a specific variable. The id can then be used to read the data directly.

In [ ]:
ids = ismn_data.get_dataset_ids(variable='soil_temperature', max_depth=1, filter_meta_dict={'lc_2005': 130, 'climate_KG': 'Csb'})
print(ids)

In [ ]:
ts, meta = ismn_data.read(ids[1], return_meta=True)
pprint(meta)
ax = ts.plot(figsize=(12,4), title=f'Time series for ID {ids[1]}')
ax.set_xlabel("Time [year]")
ax.set_ylabel("Soil Temp. [°C]")

Station locations for a specific variable can be visualised on a map. If a min/max depth is passed, only stations with a sensor that measures within the passed range are included.

In [ ]:
import cartopy.crs as ccrs
#plot available station on a map
fig, axs = plt.subplots(1, 2, figsize=(16,10), subplot_kw={'projection': ccrs.Robinson()})
ismn_data.plot_station_locations('soil_moisture', min_depth=0., max_depth=0.1, ax=axs[0])
ismn_data.plot_station_locations('soil_temperature', min_depth=0.5, ax=axs[1])
plt.show()


## Selecting and interating over data

It is often desired to iterate over all sensors that fulfill certain conditions (e.g. that measure soil moisture in a certain depth, or for a certain landcover class). For these cases the `collection` (and other components) provides iterators that take keywords and values for filtering the loaded networks/stations/sensor while iterating over single time series (of a collection, a network, or a station).

### Select  by variable and depth
In this example we iterate over all sensors in the previously loaded collection (i.e. over all active networks) that measure 'soil_moisture' in any depth (range) between 0 and 0.05 metres.

In [ ]:
for network, station, sensor in ismn_data.collection.iter_sensors(variable='soil_moisture', 
                                                                  depth=[0., 0.05]):
    data = sensor.read_data()
    display(station)
    print('\033[1m' + f'Metadata for sensor {sensor}:'+ '\033[0m')
    display(sensor.metadata.to_pd())
    ax = data.plot(figsize=(12,4), title=f'Time series for sensor {sensor.name}')
    ax.set_xlabel("Time [year]")
    ax.set_ylabel("Soil Moisture [$m^3 m^{-3}$]")
    break # for this example we stop after the first sensor

### Selecting by variable and other metadata (1)
In this example we iterate over all sensors for the network 'RMSN' and filter those that measure precipitation within an ESA CCI Landcover pixel that is marked as 'Cropland, rainfed' (10) or 'Grassland' (130).

In [ ]:
ismn_data.print_landcover_dict()

In [ ]:
for station, sensor in ismn_data.collection['RSMN'].iter_sensors(variable='precipitation', 
                                                                 filter_meta_dict={'lc_2010': [10, 130]}):
    data = sensor.read_data()
    metadata = sensor.metadata
    display(station)
    print('\033[1m' + f'Metadata for sensor {sensor}:' + '\033[0m')
    display(metadata.to_pd())
    ax = data.plot(figsize=(12,4), title=f'Time series for sensor {sensor.name}')
    ax.set_xlabel("Time [year]")
    ax.set_ylabel("Precipitation [mm]")
    break # for this example we stop after the first sensor

### Selecting by variable, depth and metadata (2)
In this example we iterate over all sensors in the collection and filter those that measure soil_moisture between 0 and 10 cm within an ESA CCI Landcover pixel that is marked as 'Cropland, rainfed' (10) or 'Grassland' (130), and has one of the follwing climate classes assigned: Csc, Cfa, Dfc. In addition we set all those soil moisture values that are **not** flagged as 'good' (G) to NaN.

In [ ]:
display(ismn_data.print_climate_dict())

In [ ]:
from ismn.meta import Depth
for network, station, sensor in ismn_data.collection \
    .iter_sensors(variable='soil_moisture',
                  depth=Depth(0.,0.05),
                  filter_meta_dict={'lc_2010': [10, 130],
                                    'climate_KG':['Csc', 'Cfa', 'Dfc']}):
    
    data = sensor.read_data()
    data.loc[data['soil_moisture_flag'] != 'G', 'soil_moisture'] = np.nan
    metadata = sensor.metadata
    display(network)
    display(station)
    print('\033[1m' + f'Metadata for sensor {sensor}:'+ '\033[0m')
    display(metadata.to_pd())
    ax = data.plot(figsize=(12,4), title=f"G-flagged SM for '{sensor.name}' at station '{station.name}' in network '{network.name}''")
    ax.set_xlabel("Time [year]")
    ax.set_ylabel("Soil Moisture [$m^3 m^{-3}$]")
    break # for this example we stop after the first sensor